In [1]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler

In [2]:
consumer_key = "LzsCOl12ZZ79FObx7BV9a1R3a"
consumer_secret = "gODxZ1kHJHDGgYfyP12AUN37UTlDd6yn3OsSV9Tl0iwu0K0cRY"
access_token = "296102538-kYIK8SULCmRK4vgvYD2DhKbwvSXrgTdTkWh0Nvsp"
access_token_secret = "Ib9obwgcE5tZk4S6tY5rAqI5iXOeJc9FGFg28qHoa4YpP"

In [3]:
auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

In [4]:
query = "Covid cases in Maharashtra"

In [5]:
Tweets = api.search( query , count=10 , lang='en' , exclude='retweets' , tweet_mode='extended' )
# tweet_mode='extended'
# tweet_mode='compat'

for tweet in Tweets:
    print(tweet,"\n")
    print(".....................................\n")

Status(_api=<tweepy.api.API object at 0x000002ED6EA2FD48>, _json={'created_at': 'Thu Oct 29 08:10:56 +0000 2020', 'id': 1321726197126946818, 'id_str': '1321726197126946818', 'full_text': 'Seeing an increase in number of COVID cases across virtually the entire state of Maharashtra for the past two days. Not sure whether this will turn out into a second wave yet. Be careful folks, maintain all precautions.', 'truncated': False, 'display_text_range': [0, 219], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 66128084, 'id_str': '66128084', 'name': 'Amit Thadhani', 'screen_name': 'amitsurg', 'location': 'Mumbai , Bharat', 'descri

In [6]:
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)
# print(df.shape)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []


In [7]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [8]:
stream(data=["Covid cases in Maharashtra"] , file_name='my_tweets')

In [9]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,Seeing an increase in number of COVID cases ac...,Amit Thadhani,76068,21287,"Mumbai , Bharat",False,0,0,2020-10-29 08:10:56
1,WEAR MASK Please\nIn order that @mumbairailuse...,Mumbai Railway Users,31130,92831,Mumbai,False,0,0,2020-10-29 08:10:55
2,Karnataka stands second only to Maharashtra in...,Frontline,7928,38958,Chennai,True,0,0,2020-10-29 05:34:43
3,RT @thesuniljain: India's Covid cases 40% lowe...,Ishaan Gera,1091,188,New Delhi,False,0,7,2020-10-29 04:48:41
4,#Coronavirus news live updates:\n\n* India's #...,Deccan Herald,244456,60702,India,True,2,0,2020-10-29 03:42:49


In [10]:
from textblob import TextBlob

In [11]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [12]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [13]:
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [14]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,Seeing an increase in number of COVID cases ac...,Amit Thadhani,76068,21287,"Mumbai , Bharat",False,0,0,2020-10-29 08:10:56,Seeing an increase in number of COVID cases ac...
1,WEAR MASK Please\nIn order that @mumbairailuse...,Mumbai Railway Users,31130,92831,Mumbai,False,0,0,2020-10-29 08:10:55,WEAR MASK Please In order that strictly follow...
2,Karnataka stands second only to Maharashtra in...,Frontline,7928,38958,Chennai,True,0,0,2020-10-29 05:34:43,Karnataka stands second only to Maharashtra in...
3,RT @thesuniljain: India's Covid cases 40% lowe...,Ishaan Gera,1091,188,New Delhi,False,0,7,2020-10-29 04:48:41,RT India s Covid cases 40 lower than a month a...
4,#Coronavirus news live updates:\n\n* India's #...,Deccan Herald,244456,60702,India,True,2,0,2020-10-29 03:42:49,Coronavirus news live updates India s Covid19 ...


In [15]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,Seeing an increase in number of COVID cases ac...,Amit Thadhani,76068,21287,"Mumbai , Bharat",False,0,0,2020-10-29 08:10:56,Seeing an increase in number of COVID cases ac...,Negative
1,WEAR MASK Please\nIn order that @mumbairailuse...,Mumbai Railway Users,31130,92831,Mumbai,False,0,0,2020-10-29 08:10:55,WEAR MASK Please In order that strictly follow...,Neutral
2,Karnataka stands second only to Maharashtra in...,Frontline,7928,38958,Chennai,True,0,0,2020-10-29 05:34:43,Karnataka stands second only to Maharashtra in...,Neutral
3,RT @thesuniljain: India's Covid cases 40% lowe...,Ishaan Gera,1091,188,New Delhi,False,0,7,2020-10-29 04:48:41,RT India s Covid cases 40 lower than a month a...,Neutral
4,#Coronavirus news live updates:\n\n* India's #...,Deccan Herald,244456,60702,India,True,2,0,2020-10-29 03:42:49,Coronavirus news live updates India s Covid19 ...,Positive


In [16]:
n = 1
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 WEAR MASK Please
In order that @mumbairailusers strictly follow #COVID  protocols to avert any resurgence in no. of… https://t.co/WlYLCKMr2d

Clean tweet:
 WEAR MASK Please In order that strictly follow COVID protocols to avert any resurgence in no of

Sentiment of the tweet:
 Neutral


In [17]:
n = 20
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @TOIMumbai: Maharashtra: 11 Covid-19 care centres shut in Latur as cases decline https://t.co/BSmGMoI5NC

Clean tweet:
 RT Maharashtra 11 Covid 19 care centres shut in Latur as cases decline

Sentiment of the tweet:
 Neutral


In [18]:
n = 203
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @AbhishBanerj: Maharashtra is still No.1 in Covid cases.

Just compare to Gujarat, which is now 16th worst affected.

Massive floods in…

Clean tweet:
 RT Maharashtra is still No 1 in Covid cases Just compare to Gujarat which is now 16th worst affected Massive floods in

Sentiment of the tweet:
 Negative
